In [ ]:
# coding:utf8
#参考：https://www.jianshu.com/p/6ca1344a09db
import re
import requests
import random
from zlib import crc32
from base64 import b64decode
import urllib
import pymongo
import pandas as pd
from os import path
import time
from lxml import etree


# 网上下载视频播放地址的方案

问题在于最终下载的视频清晰率非常低

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTHL, like Gecko) Chrome/66.0.3359.181 Safari/537.36",
}
    
def get_video_url_api(video_id='v02004190000bjqdaaq0ifkkoafl5rg0'):
    '''取视蘋地iff所在包的uri'''
    r = str(random.random())[2:]
    url_part = "/video/urls/v/1/toutiao/mp4/{}?r={}".format(video_id, r) 
    s = crc32(url_part.encode())
    url = "https://ib.365yg.com{}&s={}".format(url_part,s) 
    return url
 
def get_video_url(url):
    #获取视频地址
    resp = requests.get(url, headers=headers) 
    j_resp = resp.json()
    video_url = j_resp['data']['video_list']['video_1']['main_url'] 
    print(j_resp['data'])
    video_url = b64decode(video_url.encode()).decode() 
    return video_url
 
def get_video_id(url):
    #获取视频id
    resp = requests.get(url, headers=headers)
    # 获取video_id
    print(resp.text)
    search = re.search("\"vid\":\"([^\"]+)\",", resp.text)
    print(search.group(1))
    return search.group(1)
 
def main(video_id):
    video_url_api = get_video_url_api(video_id)
    video_url = get_video_url(video_url_api)
    return video_url

def downloadfile(filepath,url):
    r=requests.get(url)
    print("****Connected****")
    f=open(filepath,'wb');
    print("Donloading.....")
    for chunk in r.iter_content(chunk_size=255): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    print("Done")
    f.close()

In [ ]:
username_str = 'breadt'
password_str = 'Breadt@2019'

username = urllib.parse.quote_plus(username_str)
password = urllib.parse.quote_plus(password_str)

client = pymongo.MongoClient('mongodb://%s:%s@192.168.31.87:27017/' % (username, password))
db = client["ixigua"]

data = []
for x in db['video_info'].find():
    data.append(x['video_detail_info'])
    
client.close()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
for index, item in df.iterrows():
    filename = item['video_id'] + '.mp4'
    filepath = '../video/' + filename
    
    if not path.exists(filepath):
        print(item['video_id'])
        url = main(item['video_id'])
        downloadfile(filepath, url)
        time.sleep(10)


# 通过ixigua的pc端获取数据

In [25]:
url = 'https://www.ixigua.com/i6787290905767313933/?logTag=u4QC4gSEUEgaIOo85JwpA'
headers = {
    ':authority': 'www.ixigua.com',
    ':method':'GET',
    ':path': '/i6787290905767313933/?logTag=u4QC4gSEUEgaIOo85JwpA',
    ':scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7,ko;q=0.6,zh-TW;q=0.5',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'
}

# r = requests.get(url, headers=headers)
# r.text

import requests
from hyper.contrib import HTTP20Adapter
import hyper

session = requests.session()
session.mount(url, HTTP20Adapter())

r = session.get(url, headers=headers)
print(r.text)

<!DOCTYPE html>
  <html>
    <head></head>
    <body>
      <script>
        window.config = {"url":"/api/verify","retryInterval":2000,"retryTimes":1,"retryFailingStrategy":"","redirectURL":"https://www.ixigua.com/i6787290905767313933/?logTag=u4QC4gSEUEgaIOo85JwpA","key":"5040e4c1-ef89-457a-a6d1-7bc00e44f5cc","prefix":"https://pc-basic.ixigua.com","psm":"toutiao.fe.xigua_video_web_pc"};
        tac='i+2gv1fhxxynbs!i#ywos"yZl!%s"l"u&kLs#l l#vr*charCodeAtx0[!cb^i$1em7b*0d#>>>s j\uffeel  s#'
      </script>
      <script>
        !function(t,n){if("object"==typeof exports&&"object"==typeof module)module.exports=n();else if("function"==typeof define&&define.amd)define([],n);else{var r=n();for(var e in r)("object"==typeof exports?exports:t)[e]=r[e]}}(window,(function(){return function(t){var n={};function r(e){if(n[e])return n[e].exports;var o=n[e]={i:e,l:!1,exports:{}};return t[e].call(o.exports,o,o.exports,r),o.l=!0,o.exports}return r.m=t,r.c=n,r.d=function(t,n,e){r.o(t,n)||Object.defineP

In [ ]:
html = etree.HTML(r.text)

In [ ]:
html.xpath('.//script/text()')

In [ ]:
r.text